***다음 내용이 필수적으로 들어가야 합니다.***

- RNN / LSTM의 한계
- 트랜스포머의 핵심 원리
- Self - Attention
    - Query, Key, Value
    - Cross Attention vs Self Attention
    - Scaled Dot-Product Attention
- 트랜스포머의 전처리 과정
    - Positional Encoding
- 트랜스포머의 인코더/디코더 구조
    - Multi-Head Attention
        - Multi-Head Attention의 핵심 원리 및 장점
        - Masked Multi-Head Attention
    - Encoder-Decoder Attention

###넹

## RNN, LSTM 복습
- RNN, LSTM 순차데이터
- RNN: 앞뒤 내용이 서로 연관된 데이터에 대해 기억을 통해 이전 단계에서 처리한 정보를 현재 단계의 입력에 함께 활용. 모든 타임 스텝에서 동일한 가중치 반복 사용하는 가중치 공유
    - 한계: 장기 의존성 문제 - 기울기 폭발, 기울기 소실(초반부 데이터들이 후반부 데이터들에 반영되지 X)
- LSTM: 중요한 것은 오래, 필요 없는 것은 잃어버리자. 
    - 여전히 긴 문장에 있어서 Vanishing gradient 발생

## Attention 등장
- Seq2Seq: input 내용을 다른 시퀀스로 변환하는 모델 구조
    - 입,출력 단어 개수가 달라도 된다. 
    - Encoder -> Context Vector -> Decoder
    - 병목현상: "고정된 크기의 Context Vector"로 인해 입력 문장이 길어질수록 압축이 어려움 -> 일부 입력 데이터들의 소실

=> Attention: Decoder가 단어를 생성할 때마다 입력 문장 전체를 "다시 확인", 생성 단어가 "존재하는 내용"을 집중적으로 확인, "가장 관련있는 정보"만 이용하여 Context Vector 생성.

### Attention
- 중요한 것에 집중
- 가장 중요한 부분에 더 높은 가중치 부여
- Decode 때마다 context vector 생성

* 과정
- 입력을 벡터 형태로 변화하여 수치화
- 각 요소별 중요도 점수 산출
    - 현재 처리 중인 작업 기준으로 어떤 단어에 집중할 지 산출
    - 관련도 채점(관계, 유사도), 중요도 배분(SoftMax, 어텐션 가중치)
        - 벡터 간 유사성, 상관 관계, 기타 종속성을 결정(정량화 + softmax)로 가중치 계싼
- 어텐션 가중치에 따라 특정 단어의 정보에 더 집중, 덜 집중
- 최종 결과 도출

* 장점
- 시간 경과에 대한 유연성 (vs RNN)
    - 모든 단어에서 다른 모든 단어로 직접적인 연결고리를 만듦 -> 멀리 떨어져 있어도(시간, 순서 거리가 멀어도) 정보 손실 없이 명확하게 파악 가능
- 공간에 대한 유연성 (vs CNN)
    - CNN: 고정된 필터 <-> Attention: 전역적인 관계 학습 가능
- 병렬화 (vs RNN)
    - 순차계산이 필수적이지 않음. 모든 단어의 관계를 한번에, 독립적으로 계산 가능

### Attention is All You Need
- Self-Attention 방식
- RNN 기반 모델의 느린 학습 속도, 장기 의존성 문제 해결 
    -> 순차적 계산 구조 제거. 인코더 ~ 디코더까지 오직 Attention만 사용  
  
- Transformer: self-attentio 기반
- Recurrence, Convolution이 필수가 아니다! + 병렬화!
- LLM 탄생. BERT & GPT / NLP + Vision, 음성 처리, 신약 개발 등

## Transformer
- 순차 구조X, 오직 어텐션만으로 문장의 의미와 구조 파악
- Seq2Seq: RNN 사용하여 인코더, 디코더 구현
- Transformer: Attention만으로 인코더, 디코더 구현

### Self Attention
- "문장 안"에서 어떤 단어가 다른 단어들과 얼마나 중요한 관계를 맺고 있는지
- Q, K, V: 가장 관련 있는 정보를 효과적으로 찾아내기 위해 역할 분할
    - Q: Query
        - "질문", "요청": 당장 알고 싶거나 초점을 맞추고 있는 대상
        - 다른 단어들과의 관계를 파악하려 할 때, 그 기준이 되는 거 -> "나 자신"

    - K: Key
        - 검색 대상이 되는 모든 정보들이 달고 있는 "이름표, index"
        - Query가 이 키들과 자신을 비교하여 얼마나 관련 있는지 평가 진행

    - V: Value
        - 키와 한 쌍으로 묶여 있는 "실제 내용물"
        - Query + Key가 관련도 계산 -> 가장 관련 있는 key 선택 -> key에 해당하는 Value를 가져와 사용
        - EX. Quey: 서강대학교 인사이트 14기 인원, Key: 학번, Value: 학번이 Query와 가장 관련 있다고 판단되었다면, 학생 정보(나이, 학과, 수강 현황 등)

- VS 기존 어텐션 (Cross-Attention)
    - 슬쩍슬쩍 보면서 생성 <-> 문장 자체를 뜯어보며 내부 관계 파악
    - 하나의 시퀀스가 완전히 다른 시퀀스 참고하여 정보를 만듦 <-> 한 문장 내 모든 단어 끼리의 관계
    - 서로 다른 시퀀스 <-> 동일 시퀀스(문장 내부) 참조
    - 인코더-디코더 연결, 정보 병목 현상 해소 <-> 시퀀스 내부의 문맥 및 의존 관계 파악
    - 병렬처리 불가능 <-> 가능
    - 벡터 임베딩, 내정 및 정렬 함수, 중요도 배분(softmax, attention weight), 문맥이 완벽하게 반영된 새로운 벡터

- Dot-Product Attention: 쿼리와 키 벡터를 내적하여 유사도를 구함 -> Attention Score 계산
    - 문제점: 벡터의 차원이 커질수록 내적의 결과가 너무 커지거나 작아진다 -> Vanishing Gradient 문제 발생

- Scaled Dot-poduct Attention: dot-product attention 내적 진행 후, 스케일링 과정 추가 (계산된 값을  $\sqrt{d_K}$ 로 나누어 스케일링)

### Multi-Head Attention
- Self-Attention을 여러 개의 머리로 동시에, 서로 다른 관점에서 실행
- 인코더에서 한 번, 디코더에서 두 번 (첫 번째를 Masked Multi-Head Attention)
- 문장 내 단어들의 관계를 파악하기 위해서 한 단어와 다른 단어 간의 관계를 여러 차원으로 나누어 병렬로 학습. 
- 작동 방식
    EX) I Love YOU. 각 단어 512차원, 어텐션 헤드 수 8개, 각 헤드 차원 64차원
    1. 분할 - 8개의 헤드에 단어 벡터 할당. (8개의 작은 벡터 그룹 (Q,K,V)에 투영하여 8개의 서로 다른 관점으로 분할)
    2. 병렬 어텐션 계산 - 각 헤드들이 각자 분석
        - 각 헤드의 관점에서 문맥을 이해한 결과 벡터를 출력
    3. 결합 및 최종 투영 - 전문가들의 분석 결과 취합
        - 8개 헤드의 8개 출력을 하나로 합쳐서 최종 출력

### Positional Encoding
- 단어의 위치 정보를 벡터에 추가하여 각 단어가 문장의 몇번째 위치하는지 알 수 있다.
- "문법적 측면", "문맥 파악"을 위해서 등장 순서가 매우 중요

### 전체 아키텍처
- 전처리
    - 입력: 텍스트 -> 텍스트 토큰화 -> 임베딩(숫자 벡터로 변환, Word2Vec) -> Positional Encoding으로 순서 정보 추가 -> 인코더 레이어...

- Encoder: 입력 문장 이해 및 요약된 의미 벡터로 변환
    - Multi-Head Attention Layer ~ 단어와 다른 단어 관계 파악. (score, weight)
    - Feed-Forward Layer: 입력 벡터의 차원 확장 + 비선형 변환 적용
    - 잔차 연결: 입력 값을 그대로 출력에 더하면서 Layer 연결 + 층 정규화
-> 입력 문장의 문맥적 의미 깊이 이해


- Decoder: Encoder가 분석한 입력 문장의 의미 벡터를 받아서 출력 문장을 순차적으로 생성
    - Masked Multi-Head Attention
        : 미래 시점의 단어 정보를 참고하지 못하도록 마스크를 적용하는 어텐션 메커니즘, 과거 정보만으로 다음 단어 예측하도록 강제
    - Encoder-Decoder Multi-Head Attention (Cross-Attention)
        : 인코더의 출력을 참고하면서 현재 생성 중인 단어를 입력 문장의 의미와 연결
    - Feed-Forward Layer